In [61]:
# A 165 LINE TOPOLOGY OPTIMIZATION CODE BY NIELS AAGE AND VILLADS EGEDE JOHANSEN, JANUARY 2013
from __future__ import division
import numpy as np
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import spsolve
from matplotlib import colors
import matplotlib.pyplot as plt
from threading import Thread
# MAIN DRIVER
def topopt(nelx,nely,volfrac,penal,rmin,ft, maxiter, minchange, f, fixed, colors, shouldplot=True):
	Emin=1e-9
	Emax=1.0
	ndof = 2*(nelx+1)*(nely+1)
	# Allocate design variables (as array), initialize and allocate sens.
	x=volfrac * np.ones(nely*nelx,dtype=float)
	xold=x.copy()
	xPhys=x.copy()
	g=0 # must be initialized to use the NGuyen/Paulino OC approach
	dc=np.zeros((nely,nelx), dtype=float)
	# FE: Build the index vectors for the for coo matrix format.
	KE=lk()
	edofMat=np.zeros((nelx*nely,8),dtype=int)
	for elx in range(nelx):
		for ely in range(nely):
			el = ely+elx*nely
			n1=(nely+1)*elx+ely
			n2=(nely+1)*(elx+1)+ely
			edofMat[el,:]=np.array([2*n1+2, 2*n1+3, 2*n2+2, 2*n2+3,2*n2, 2*n2+1, 2*n1, 2*n1+1])
	# Construct the index pointers for the coo format
	iK = np.kron(edofMat,np.ones((8,1))).flatten()
	jK = np.kron(edofMat,np.ones((1,8))).flatten()    
	# Filter: Build (and assemble) the index+data vectors for the coo matrix format
	nfilter=int(nelx*nely*((2*(np.ceil(rmin)-1)+1)**2))
	iH = np.zeros(nfilter)
	jH = np.zeros(nfilter)
	sH = np.zeros(nfilter)
	cc=0
	for i in range(nelx):
		for j in range(nely):
			row=i*nely+j
			kk1=int(np.maximum(i-(np.ceil(rmin)-1),0))
			kk2=int(np.minimum(i+np.ceil(rmin),nelx))
			ll1=int(np.maximum(j-(np.ceil(rmin)-1),0))
			ll2=int(np.minimum(j+np.ceil(rmin),nely))
			for k in range(kk1,kk2):
				for l in range(ll1,ll2):
					col=k*nely+l
					fac=rmin-np.sqrt(((i-k)*(i-k)+(j-l)*(j-l)))
					iH[cc]=row
					jH[cc]=col
					sH[cc]=np.maximum(0.0,fac)
					cc=cc+1
	# Finalize assembly and convert to csc format
	H=coo_matrix((sH,(iH,jH)),shape=(nelx*nely,nelx*nely)).tocsc()	
	Hs=H.sum(1)
	# BC's and support
	dofs=np.arange(2*(nelx+1)*(nely+1)) #Liste de chacuns des éléments
	free=np.setdiff1d(dofs,fixed)
	# Solution and RHS vectors
	u=np.zeros((ndof,1))
	# Initialize plot and plot the initial design
   	# Set loop counter and gradient vectors 
	loop=0
	change=1
	dv = np.ones(nely*nelx)
	dc = np.ones(nely*nelx)
	ce = np.ones(nely*nelx)
	while change>minchange and loop<maxiter:
		loop=loop+1
		#print(loop, end=" ")
		# Setup and solve FE problem
		sK=((KE.flatten()[np.newaxis]).T*(Emin+(xPhys)**penal*(Emax-Emin))).flatten(order='F')
		K = coo_matrix((sK,(iK,jK)),shape=(ndof,ndof)).tocsc()
		# Remove constrained dofs from matrix
		K = K[free,:][:,free]
		# Solve system 
		u[free,0]=spsolve(K,f[free,0])    
		# Objective and sensitivity
		ce[:] = (np.dot(u[edofMat].reshape(nelx*nely,8),KE) * u[edofMat].reshape(nelx*nely,8) ).sum(1)
		obj=( (Emin+xPhys**penal*(Emax-Emin))*ce ).sum()
		dc[:]=(-penal*xPhys**(penal-1)*(Emax-Emin))*ce
		dv[:] = np.ones(nely*nelx)
		# Sensitivity filtering:
		if ft==0:
			dc[:] = np.asarray((H*(x*dc))[np.newaxis].T/Hs)[:,0] / np.maximum(0.001,x)
		elif ft==1:
			dc[:] = np.asarray(H*(dc[np.newaxis].T/Hs))[:,0]
			dv[:] = np.asarray(H*(dv[np.newaxis].T/Hs))[:,0]
		# Optimality criteria
		xold[:]=x
		(x[:],g)=oc(nelx,nely,x,volfrac,dc,dv,g)
		# Filter design variables
		if ft==0:   xPhys[:]=x
		elif ft==1:	xPhys[:]=np.asarray(H*x[np.newaxis].T/Hs)[:,0]
		# Compute the change by the inf. norm
		change=np.linalg.norm(x.reshape(nelx*nely,1)-xold.reshape(nelx*nely,1),np.inf)
		# Write iteration history to screen (req. Python 2.6 or newer)
		"""print("it.: {0} , obj.: {1:.3f} Vol.: {2:.3f}, ch.: {3:.3f}".format(\
					loop,obj,(g+volfrac*nelx*nely)/(nelx*nely),change))"""
	# Make sure the plot stays and that the shell remains
		xPhys_2d = xPhys.reshape((nelx, nely))
		xPhys_2d = np.interp(xPhys_2d, (xPhys_2d.min(), xPhys_2d.max()), (0, 255)).astype(int)
		color_grid = [[(val, val, val) for val in row] for row in xPhys_2d]
		for i in range(nelx):
			for j in range(nely):
				colors[i][j] = color_grid[i][j]
#element stiffness matrix
def lk():
	E=1
	nu=0.3
	k=np.array([1/2-nu/6,1/8+nu/8,-1/4-nu/12,-1/8+3*nu/8,-1/4+nu/12,-1/8-nu/8,nu/6,1/8-3*nu/8])
	KE = E/(1-nu**2)*np.array([ [k[0], k[1], k[2], k[3], k[4], k[5], k[6], k[7]],
	[k[1], k[0], k[7], k[6], k[5], k[4], k[3], k[2]],
	[k[2], k[7], k[0], k[5], k[6], k[3], k[4], k[1]],
	[k[3], k[6], k[5], k[0], k[7], k[2], k[1], k[4]],
	[k[4], k[5], k[6], k[7], k[0], k[1], k[2], k[3]],
	[k[5], k[4], k[3], k[2], k[1], k[0], k[7], k[6]],
	[k[6], k[3], k[4], k[1], k[2], k[7], k[0], k[5]],
	[k[7], k[2], k[1], k[4], k[3], k[6], k[5], k[0]] ]);
	return (KE)
# Optimality criterion
def oc(nelx,nely,x,volfrac,dc,dv,g):
	l1=0
	l2=1e9
	move=0.2
	# reshape to perform vector operations
	xnew=np.zeros(nelx*nely)
	while (l2-l1)/(l1+l2)>1e-3:
		lmid=0.5*(l2+l1)
		xnew[:]= np.maximum(0.0,np.maximum(x-move,np.minimum(1.0,np.minimum(x+move,x*np.sqrt(-dc/dv/lmid)))))
		gt=g+np.sum((dv*(xnew-x)))
		if gt>0 :
			l1=lmid
		else:
			l2=lmid
	return (xnew,gt)

In [62]:
def print_not_zero(tab):
    for t in range(len(tab)):
        if tab[t] != 0:
            print(t , " : " , tab[t])

In [63]:
nelx=120
nely=50
volfrac=0.4
rmin=3
penal=3.0
ft=1 # ft==0 -> sens, ft==1 -> dens
maxiter=100
minchange=0.01
dofs=np.arange(2*(nelx+1)*(nely+1))
ndof = 2*(nelx+1)*(nely+1)
f=np.zeros((ndof,1))
volfrac=0.4
fixed=np.array([])

In [64]:
import pygame
import sys
from pygame import gfxdraw
from PIL import Image
import ctypes
import threading
myappid = 'lisv.topopt' 
ctypes.windll.shell32.SetCurrentProcessExplicitAppUserModelID(myappid)
PADDING = 30
ROWS, COLS = nely, nelx
WIDTH, HEIGHT = COLS*10, ROWS*10
BUTTON_SIZE = 45
NODE_RADIUS = 4
CLICK_COLOR = (255, 0, 0)
HAS_CONVERGED = False
COLORS=[]
# Initialize pygame
pygame.init()

# Create the display
win = pygame.display.set_mode((WIDTH+4*PADDING+BUTTON_SIZE, HEIGHT+2*PADDING))
logo = pygame.image.load("logo.png")
pygame.display.set_caption("TOPOPT Interactif - LISV")
pygame.display.set_icon(logo)
# Node class
class Node:
    def __init__(self, x, y, num):
        self.nn = num
        self.x = x
        self.y = y
        self.forces = [0, 0, 0, 0]
        self.clicked = False
        self.dof = [False, False]
    def draw_vertical_down_arrow(self, win, color):
        for i in range(-1, 1, 1):
            pygame.draw.aaline(win, color, (self.x+i+PADDING, self.y+PADDING-10), (self.x+i+PADDING, self.y+PADDING))
            pygame.draw.aaline(win, color, (self.x+3+i+PADDING, self.y-4+PADDING), (self.x+i+PADDING, self.y+PADDING))
            pygame.draw.aaline(win, color, (self.x-3+i+PADDING, self.y-4+PADDING), (self.x+i+PADDING, self.y+PADDING))
    def draw_vertical_up_arrow(self, win, color):
        for i in range(-1, 1, 1):
            pygame.draw.aaline(win, color, (self.x+i+PADDING, self.y+PADDING+10), (self.x+i+PADDING, self.y+PADDING))
            pygame.draw.aaline(win, color, (self.x+3+i+PADDING, self.y+4+PADDING), (self.x+i+PADDING, self.y+PADDING))
            pygame.draw.aaline(win, color, (self.x-3+i+PADDING, self.y+4+PADDING), (self.x+i+PADDING, self.y+PADDING))
    def draw_horizontal_left_arrow(self, win, color):
        for i in range(-1, 1, 1):
            pygame.draw.aaline(win, color, (self.x+10+PADDING, self.y+i+PADDING), (self.x+PADDING, self.y+i+PADDING))
            pygame.draw.aaline(win, color, (self.x+PADDING, self.y+i+PADDING), (self.x+3+PADDING, self.y-3+i+PADDING))
            pygame.draw.aaline(win, color, (self.x+PADDING, self.y+i+PADDING), (self.x+3+PADDING, self.y+3+i+PADDING))
    def draw_horizontal_right_arrow(self, win, color):
        for i in range(-1, 1, 1):
            pygame.draw.aaline(win, color, (self.x+PADDING-10, self.y+i+PADDING), (self.x+PADDING, self.y+i+PADDING))
            pygame.draw.aaline(win, color, (self.x+PADDING, self.y+i+PADDING), (self.x-3+PADDING, self.y-3+i+PADDING))
            pygame.draw.aaline(win, color, (self.x+PADDING, self.y+i+PADDING), (self.x-3+PADDING, self.y+3+i+PADDING))
    def draw_y_support(self, win, color):
        x, y = self.x, self.y
        signes=[-1, 1]
        s=signes[int(round(y/HEIGHT))]
        pygame.draw.polygon(win,(100,100,100),((x+PADDING,y+PADDING),(x-5+PADDING,y+s*10+PADDING),(x+5+PADDING,y+s*10+PADDING)))
        gfxdraw.aapolygon(win,((x+PADDING,y+PADDING),(x-5+PADDING,y+s*10+PADDING),(x+5+PADDING,y+s*10+PADDING)),(0,0,0))
        gfxdraw.filled_circle(win, x-3+PADDING, y+s*13+PADDING, 2,  (100, 100, 100))
        gfxdraw.aacircle(win, x-3+PADDING, y+s*13+PADDING, 2,  (0, 0, 0))
        gfxdraw.filled_circle(win, x+3+PADDING, y+s*13+PADDING, 2,  (100, 100, 100))
        gfxdraw.aacircle(win, x+3+PADDING, y+s*13+PADDING, 2,  (0, 0, 0))
    def draw_x_support(self, win, color):
        x, y = self.x, self.y
        signes=[-1, 1]
        s=signes[int(round(x/WIDTH))]
        pygame.draw.polygon(win,(100,100,100),((x+PADDING,y+PADDING),(x+s*10+PADDING,y-5+PADDING),(x+s*10+PADDING,y+5+PADDING)))
        gfxdraw.aapolygon(win,((x+PADDING,y+PADDING),(x+s*10+PADDING,y-5+PADDING),(x+s*10+PADDING,y+5+PADDING)),(0,0,0))
        gfxdraw.filled_circle(win, x+s*13+PADDING, y+3+PADDING, 2,  (100, 100, 100))
        gfxdraw.aacircle(win, x+s*13+PADDING, y+3+PADDING, 2,  (0, 0, 0))
        gfxdraw.filled_circle(win, x+s*13+PADDING, y-3+PADDING, 2,  (100, 100, 100))
        gfxdraw.aacircle(win, x+s*13+PADDING, y-3+PADDING, 2,  (0, 0, 0))
    def is_over(self, pos):
        distance = ((pos[0] - self.x)**2 + (pos[1] - self.y)**2)**0.5
        return distance <= NODE_RADIUS

# Create nodes
node_dist_x = WIDTH // (COLS - 1)
node_dist_y = HEIGHT // (ROWS - 1)
nodes = [Node(i*node_dist_x, j*node_dist_y, i*(nely+1)+j) for j in range(ROWS+1) for i in range(COLS+1)]
tool_number = 0
def main():
    clock = pygame.time.Clock()
    global tool_number,fixed, HAS_CONVERGED
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                pos = pygame.mouse.get_pos()
                x, y = pos
                pos = (pos[0]-PADDING, pos[1]-PADDING)
                buttons = [(WIDTH + 2 *PADDING, PADDING + i * (BUTTON_SIZE + PADDING), BUTTON_SIZE, BUTTON_SIZE) for i in range(7)]
                
                for j, button in enumerate(buttons):

                    if button[0] <= x and x <= button[0]+BUTTON_SIZE:
                        if button[1] <= y and y <= button[1]+BUTTON_SIZE:
                            if j < 6:
                                tool_number = j
                            else:
                                for node in nodes:
                                    f[2*node.nn]=node.forces[0]+node.forces[2]
                                    f[2*node.nn+1]=node.forces[1]+node.forces[3]
                                    if node.dof[0]:
                                        fixed=np.union1d(fixed,np.array(2*node.nn))
                                    if node.dof[1]:
                                        fixed=np.union1d(fixed,np.array(2*node.nn+1))
                                HAS_CONVERGED=True
                                COLORS = [[(0,0,0) for i in range(nely)] for j in range(nelx)]
                                x = threading.Thread(target=topopt, args=(nelx,nely,volfrac,penal,rmin,ft, maxiter, minchange, f, fixed, COLORS))
                                x.start()
                                image = Image.new('RGB', (nelx, nely))
                                pixels = image.load()
                                for x in range(0, nelx):
                                    for y in range(0, nely):
                                        r, g, b = COLORS[x][y]
                                        if (r+g+b) > 100:
                                            pixels[x,y] = (255,255,255)
                                        else:
                                            pixels[x,y] = (0,0,0)
                                image.save("gen.png")
                for node in nodes:
                    if node.is_over(pos):
                        if tool_number < 4:
                            node.forces[tool_number]=1
                        elif tool_number == 4:
                            node.dof[1]=True
                        elif tool_number == 5:
                            node.dof[0]=True
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_UP:
                    tool_number=3
                if event.key == pygame.K_DOWN:
                    tool_number=1
                if event.key == pygame.K_RIGHT:
                    tool_number=0
                if event.key == pygame.K_LEFT:
                    tool_number=2
                if event.key == pygame.K_y:
                    tool_number=4
                if event.key == pygame.K_x:
                    tool_number=5
                if event.key == pygame.K_g:
                    for node in nodes:
                        f[2*node.nn]=node.forces[0]+node.forces[2]
                        f[2*node.nn+1]=node.forces[1]+node.forces[3]
                        if node.dof[0]:
                            fixed=np.union1d(fixed,np.array(2*node.nn))
                        if node.dof[1]:
                            fixed=np.union1d(fixed,np.array(2*node.nn+1))
                    HAS_CONVERGED=True
                    COLORS = [[(0, 0, 0) for i in range(nely)] for j in range(nelx)]
                    x = threading.Thread(target=topopt, args=(nelx,nely,volfrac,penal,rmin,ft, maxiter, minchange, f, fixed, COLORS))
                    x.start()
                    image = Image.new('RGB', (nelx, nely))
                    pixels = image.load()
                    for x in range(0, nelx):
                        for y in range(0, nely):
                            r, g, b = COLORS[x][y]
                            if (r+g+b) > 100:
                                pixels[x,y] = (255,255,255)
                            else:
                                pixels[x,y] = (0,0,0)
                    image.save("gen.png")
        win.fill((255, 255, 255))

        if HAS_CONVERGED:
            for x in range(0, nelx):
                for y in range(0, nely):
                    r, g, b = COLORS[x][y]
                    r = 255-r
                    g = 255-g
                    b = 255-b
                    pygame.draw.rect(win, (r, g, b), (PADDING+x*node_dist_x, PADDING+y*node_dist_y, node_dist_x, node_dist_y))
        # Draw grid
        for x in range(0, WIDTH+node_dist_x, node_dist_x):
            pygame.draw.line(win, (200, 200, 200), (x+PADDING, PADDING), (x+PADDING, HEIGHT+PADDING))
        for y in range(0, HEIGHT+node_dist_y, node_dist_y):
            pygame.draw.line(win, (200, 200, 200), (PADDING, y+PADDING), (WIDTH+PADDING, y+PADDING))
        # Draw nodes
        buttons = [(WIDTH + 2 * PADDING, PADDING + i * (BUTTON_SIZE + PADDING), BUTTON_SIZE, BUTTON_SIZE) for i in range(7)]
        for j, button in enumerate(buttons):
            pygame.draw.rect(win, (240, 240, 240), button)
            pygame.draw.rect(win, (200, 200, 200),button, width=2)
            x, y = button[0]-5, button[1]+5
            if j == 1:
                for i in range(-1, 1, 1):
                    pygame.draw.aaline(win, (255,0,0), (x+i+PADDING, y+PADDING-20), (x+i+PADDING, y+PADDING))
                    pygame.draw.aaline(win, (255,0,0), (x+6+i+PADDING, y-8+PADDING), (x+i+PADDING, y+PADDING))
                    pygame.draw.aaline(win, (255,0,0), (x-6+i+PADDING, y-8+PADDING), (x+i+PADDING, y+PADDING))
            if j == 3:
                for i in range(-1, 1, 1):
                    pygame.draw.aaline(win, (255,0,0), (x+i+PADDING, y+PADDING-20), (x+i+PADDING, y+PADDING))
                    pygame.draw.aaline(win, (255,0,0), (x+6+i+PADDING, y-12+PADDING), (x+i+PADDING, y-20+PADDING))
                    pygame.draw.aaline(win, (255,0,0), (x-6+i+PADDING, y-12+PADDING), (x+i+PADDING, y-20+PADDING))
            if j == 0:
                for i in range(-1, 1, 1):
                    pygame.draw.aaline(win, (255,0,0), (x+3+PADDING-13, y+PADDING-10+i), (x+3+PADDING+7, y+PADDING-10+i))
                    pygame.draw.aaline(win, (255,0,0), (x+3+PADDING+7, y+PADDING-10+i), (x+3+PADDING-2, y-18+PADDING+i))
                    pygame.draw.aaline(win, (255,0,0), (x+3+PADDING+7, y+PADDING-10+i), (x+3+PADDING-2, y-2+PADDING+i))
            if j == 2:
                for i in range(-1, 1, 1):
                    pygame.draw.aaline(win, (255,0,0), (x+3+PADDING-13, y+PADDING-10+i), (x+3+PADDING+7, y+PADDING-10+i))
                    pygame.draw.aaline(win, (255,0,0), (x+3+PADDING-13, y+PADDING-10+i), (x+3+PADDING-2, y-18+PADDING+i))
                    pygame.draw.aaline(win, (255,0,0), (x+3+PADDING-13, y+PADDING-10+i), (x+3+PADDING-2, y-2+PADDING+i))
            if j == 5:
                s=1
                x-=16
                y-=13
                pygame.draw.polygon(win,(100,100,100),((x+PADDING,y+PADDING),(x+s*20+PADDING,y-10+PADDING),(x+s*20+PADDING,y+10+PADDING)))
                gfxdraw.aapolygon(win,((x+PADDING,y+PADDING),(x+s*20+PADDING,y-10+PADDING),(x+s*20+PADDING,y+10+PADDING)),(0,0,0))
                gfxdraw.filled_circle(win, x+s*24+PADDING, y+6+PADDING, 4,  (100, 100, 100))
                gfxdraw.aacircle(win, x+s*24+PADDING, y+6+PADDING, 4,  (0, 0, 0))
                gfxdraw.filled_circle(win, x+s*24+PADDING, y-6+PADDING, 4,  (100, 100, 100))
                gfxdraw.aacircle(win, x+s*24+PADDING, y-6+PADDING, 4,  (0, 0, 0))
            if j == 4:
                s=1
                x-=16
                y-=13
                pygame.draw.polygon(win,(100,100,100),((x+s*24+PADDING-20, y+8+PADDING),(x+s*24+PADDING, y+8+PADDING),(x+s*24-8+PADDING,y-10+PADDING)))
                gfxdraw.aapolygon(win,((x+s*24+PADDING-20, y+8+PADDING),(x+s*24+PADDING, y+8+PADDING),(x+s*24-8+PADDING,y-10+PADDING)),(0,0,0))
                gfxdraw.filled_circle(win, x+s*24+PADDING-16, y+12+PADDING, 4,  (100, 100, 100))
                gfxdraw.aacircle(win, x+s*24+PADDING-16, y+12+PADDING, 4,  (0, 0, 0))
                gfxdraw.filled_circle(win, x+s*24+PADDING-4, y+12+PADDING, 4,  (100, 100, 100))
                gfxdraw.aacircle(win, x+s*24+PADDING-4, y+12+PADDING, 4,  (0, 0, 0))
            if j == 6:
                pygame.draw.polygon(win,(100,200,50),((x-18+PADDING,y+PADDING),(x-18+PADDING,y-20+PADDING),(x-18+s*20+PADDING,y-10+PADDING)))
                gfxdraw.aapolygon(win,((x-18+PADDING,y+PADDING),(x-18+PADDING,y-20+PADDING),(x-18+s*20+PADDING,y-10+PADDING)),(50,100,25))
                pygame.draw.polygon(win,(100,200,50),((x-6+PADDING,y+PADDING),(x-6+PADDING,y-20+PADDING),(x-6+s*20+PADDING,y-10+PADDING)))
                gfxdraw.aapolygon(win,((x-6+PADDING,y+PADDING),(x-6+PADDING,y-20+PADDING),(x-6+s*20+PADDING,y-10+PADDING)),(50,100,25))
        pos = pygame.mouse.get_pos()
        npos=(pos[0]-PADDING, pos[1]-PADDING)
        for node in nodes:
            if node.forces[0] != 0:
                node.draw_horizontal_right_arrow(win, CLICK_COLOR)
            if node.forces[1] != 0:
                node.draw_vertical_down_arrow(win, CLICK_COLOR)
            if node.forces[2] != 0:
                node.draw_horizontal_left_arrow(win, CLICK_COLOR)
            if node.forces[3] > 0:
                node.draw_vertical_up_arrow(win, CLICK_COLOR)
            if node.dof[1]:
                node.draw_y_support(win, CLICK_COLOR)
            if node.dof[0]:
                node.draw_x_support(win, CLICK_COLOR)
            #Tracer l'aperçu
            if node.is_over(npos):
                if tool_number == 0:
                    node.draw_horizontal_right_arrow(win, CLICK_COLOR)
                elif tool_number == 1:
                    node.draw_vertical_down_arrow(win, CLICK_COLOR)
                elif tool_number == 2:
                    node.draw_horizontal_left_arrow(win, CLICK_COLOR)
                elif tool_number == 3:
                    node.draw_vertical_up_arrow(win, CLICK_COLOR)
                elif tool_number == 4:
                    node.draw_y_support(win, CLICK_COLOR)
                elif tool_number == 5:
                    node.draw_x_support(win, CLICK_COLOR)
        pygame.display.update()
        clock.tick(60)

if __name__ == "__main__":
    main()

SystemExit: 